There are three types of recommendation engines-:

1. Popularity Based

2. content based 

3. Collabrative (User-Based) -: It is done by deep tensorflow engines

In [ ]:
import numpy as np
import pandas as pd 
import json
import ast 

from scipy.cluster.vq import vq,kmeans,whiten

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle 

from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Reshape, Dot,concatenate,Dropout,Dense,Lambda
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

#*Data Preprocessing*
preprocessing done you can directly load the saved data 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /data/tmdb_5000_movies.csv')

In [ ]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
data.columns 

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')




working on genres

In [ ]:
genres = (data['genres'])

In [ ]:
genres.head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [ ]:
def movie_vocab_genre(genre_list):
  all_genre=[] 
  for user_genre in ast.literal_eval(genre_list):
    all_genre.append((user_genre['name'].lower().strip()))
  return set(all_genre)

def create_vocab_genre(genres_list):
  all_genres=set()
  for genre_list in genres_list:
    all_genres|=movie_vocab_genre(genre_list)
  return list(all_genres)

In [ ]:
all_genres = create_vocab_genre(genres) 

In [ ]:
all_genres.sort()

In [ ]:
print(all_genres)

['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'foreign', 'history', 'horror', 'music', 'mystery', 'romance', 'science fiction', 'thriller', 'tv movie', 'war', 'western']


creating simple genres id

In [ ]:
genres_id = list(np.arange(len(all_genres)))
print(genres_id)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
popularity = list(data['popularity'])
ratings = list(data['vote_average'])
vote_count=list(data['vote_count'])
movie_name =  list(data['title'])

In [ ]:
print(len(popularity))

4803


#**simple recommender system(Using K-Means)**

create data

In [ ]:
cluster_data = np.vstack((np.array(popularity),np.array(ratings),np.array(vote_count)))

In [ ]:
cluster_data=cluster_data.T

In [ ]:
print(cluster_data.shape)

(4803, 3)


In [ ]:
def top_movies(cluster_data,taken):
  #normalizing the data 
  cluster_data = whiten(cluster_data)
  centroids,_ = kmeans(cluster_data,2)
  clx,_ = vq(cluster_data,centroids)
  print(clx)
  
  #choose the small cluster as it contain top movies
  choose_cluster_id = 0
  if(sum(clx) < (len(clx)-sum(clx))):
    choose_cluster_id=1
  
  #finding the nearest value
  nearest = np.ones(len(popularity)) * 1000
  for count,id in enumerate(clx):
    if(id==choose_cluster_id):
      nearest[count]=np.min((np.sum(centroids-cluster_data[count],axis=1))**2)

  final = []
  for count,value in enumerate(nearest):
    temp=dict()
    if value==1000:
      continue
    else:
      temp['name'] = movie_name[taken[count]]
      temp['value'] = value
      final.append(temp)
  final = sorted(final,key=lambda d:d['value'],reverse=True)
    
  return final


In [ ]:
final=top_movies(cluster_data,np.arange(len(movie_name)))

[1 1 1 ... 0 0 0]


Listing out top 15 movies

In [ ]:
for i in range(15):
  print((final[i])['name'])

Interstellar
Minions
Deadpool
Guardians of the Galaxy
Mad Max: Fury Road
Jurassic World
Inception
The Dark Knight
Pirates of the Caribbean: The Curse of the Black Pearl
Avatar
The Avengers
Fight Club
The Lord of the Rings: The Fellowship of the Ring
The Shawshank Redemption
Captain America: Civil War


**Recommendation based on particlar genre**

In [ ]:
print(all_genres)

['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'foreign', 'history', 'horror', 'music', 'mystery', 'romance', 'science fiction', 'thriller', 'tv movie', 'war', 'western']


In [ ]:
print(genres_id)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
def evaluate_genre(genre_list,all_genres):
  genre_one_hot = np.zeros(len(all_genres))
  for user_genre in ast.literal_eval(genre_list):
    genre_one_hot[all_genres.index(user_genre['name'].lower().strip())]=1
  return genre_one_hot

def get_movie_genre(genres_list,all_genres):
  movies_genre = np.zeros((len(genres_list),len(all_genres)))
  for count,genre_list in enumerate(genres_list):
    movies_genre[count,:] = evaluate_genre(genre_list,all_genres)
  return movies_genre

In [ ]:
movies_genre = get_movie_genre(genres,all_genres)

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
a=np.delete(a,1,0)

In [ ]:
print(a)

[[1 2 3]]


In [ ]:
def get_genre_based_clustered_data(cluster_data,index,movies_genre):
  
  indexes=[]
  taken=[]
  for i in range((cluster_data.shape)[0]):
    if movies_genre[i,index] == 0:
      indexes.append(i)
    else:
      taken.append(i)
  final_data = np.delete(cluster_data,indexes,0)
  return final_data,taken


In [ ]:
index = all_genres.index('romance')
final_data,taken =  get_genre_based_clustered_data(cluster_data,index,movies_genre)


In [ ]:
final=top_movies(final_data,taken)
for i in range(15):
  print((final[i])['name'])

[0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 0
 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 1
 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0
 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0
 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0
 0 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1
 0 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0
 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 0 1 1 1 1 1 0 1 

0.0


#**Content Based Recommender sytem**

**Recommendation based on description**

In [ ]:
import nltk
nltk.download('punkt')
import string
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
lemmer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


preparing data using overview and tagline

In [ ]:
data['description'] = data['overview'] + data['tagline']
data['description']=data['description'].fillna(' ')
description = list(data['description'].apply(lambda x:" ".join(LemNormalize(x))))

In [ ]:
description_encoded = list(map(sbert_model.encode,description))

In [ ]:
# GRADED FUNCTION: cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u * u))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v * v))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / (norm_u * norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

In [ ]:
def get_recommendation(title):
  similarity = []
  try:
    sim_input = description_encoded[movie_name.index(title)]
  except:
    sim_input = " ".join(LemNormalize(title))
    sim_input = sbert_model.encode(sim_input)

  for count,compare in enumerate(description_encoded):
    sim = cosine_similarity(sim_input,compare)
    temp = {'name':movie_name[count],'similarity':sim}
    similarity.append(temp)

  top_30 = (sorted(similarity,key=lambda d:d['similarity'],reverse=True))[:30]
  return top_30
  

In [ ]:
top_30 = get_recommendation('The Amazing Spider-Man')

In [ ]:
index= []
for movie in top_30:
  index.append(movie_name.index(movie['name']))

In [ ]:
new_cluster_data =  cluster_data[index,:]

In [ ]:
top_30=top_movies(new_cluster_data,index)

[1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


In [ ]:
print(top_30)

[{'name': 'Back to the Future', 'value': 5.553177321391432}, {'name': 'Spider-Man 2', 'value': 3.0035370368624306}, {'name': 'TRON: Legacy', 'value': 2.4712596555116053}, {'name': 'The Amazing Spider-Man', 'value': 1.816865860981024}, {'name': 'Good Will Hunting', 'value': 0.662066594150581}, {'name': 'Indiana Jones and the Last Crusade', 'value': 0.17846269484572916}, {'name': 'The Amazing Spider-Man 2', 'value': 6.5736378676669e-05}]


In [ ]:
for mov in top_30:
  print(mov['name'])

Back to the Future
Spider-Man 2
TRON: Legacy
The Amazing Spider-Man
Good Will Hunting
Indiana Jones and the Last Crusade
The Amazing Spider-Man 2


#**Recommendation based on cast crew keywords**

In [ ]:
credits=pd.read_csv('/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /data/tmdb_5000_credits.csv')

In [ ]:
credits.columns                    

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [ ]:
merged_data = pd.merge(data,credits,left_on='id',right_on='movie_id')

merged data pre-processing

In [ ]:
merged_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew'],
      dtype='object')

In [ ]:
''' We are going to use  Keywords , cast , Director(we will use director three times so to increase the weight) , genre'''

' We are going to use  Keywords , cast , Director(we will use director three times so to increase the weight) , genre'

In [ ]:
merged_data['crew']  = merged_data['crew'] .apply(ast.literal_eval)
merged_data['keywords']  = merged_data['keywords'] .apply(ast.literal_eval)
merged_data['genres']  = merged_data['genres'] .apply(ast.literal_eval)
merged_data['cast']  = merged_data['cast'] .apply(ast.literal_eval)

Director

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
merged_data['director'] = merged_data['crew'].apply(get_director)

In [ ]:
merged_data['director'] = merged_data['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
merged_data['director'] = merged_data['director'].apply(lambda x: [x,x, x])

In [ ]:
merged_data['director'].head(18)

0            [jamescameron, jamescameron, jamescameron]
1         [goreverbinski, goreverbinski, goreverbinski]
2                     [sammendes, sammendes, sammendes]
3     [christophernolan, christophernolan, christoph...
4         [andrewstanton, andrewstanton, andrewstanton]
5                        [samraimi, samraimi, samraimi]
6               [byronhoward, byronhoward, byronhoward]
7                  [josswhedon, josswhedon, josswhedon]
8                  [davidyates, davidyates, davidyates]
9                  [zacksnyder, zacksnyder, zacksnyder]
10              [bryansinger, bryansinger, bryansinger]
11              [marcforster, marcforster, marcforster]
12        [goreverbinski, goreverbinski, goreverbinski]
13        [goreverbinski, goreverbinski, goreverbinski]
14                 [zacksnyder, zacksnyder, zacksnyder]
15        [andrewadamson, andrewadamson, andrewadamson]
16                 [josswhedon, josswhedon, josswhedon]
17              [robmarshall, robmarshall, robma

Keywords

In [ ]:
merged_data['keywords'].head(1)

0    [{'id': 1463, 'name': 'culture clash'}, {'id':...
Name: keywords, dtype: object

In [ ]:
merged_data['keywords'] = merged_data['keywords'].apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

In [ ]:
merged_data['keywords'].head(1)

0    [culture clash, future, space war, space colon...
Name: keywords, dtype: object

In [ ]:
def LemNormalize_keywords(x):
  return list(map(LemNormalize,x))

In [ ]:
merged_data['keywords'] = merged_data['keywords'].apply(lambda x:list(map(LemNormalize,x)))
merged_data['keywords'] = merged_data['keywords'].apply(lambda x:list(map(" ".join,x)))

In [ ]:
merged_data['keywords'].iloc[1]

['ocean',
 'drug abuse',
 'exotic island',
 'east india trading company',
 'love of one life',
 'traitor',
 'shipwreck',
 'strong woman',
 'ship',
 'alliance',
 'calypso',
 'afterlife',
 'fighter',
 'pirate',
 'swashbuckler',
 'aftercreditsstinger']

cast

In [ ]:
merged_data['cast'] = merged_data['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
merged_data['cast'] = merged_data['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

genre

In [ ]:
merged_data['genres'].iloc[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
merged_data['genres'] = merged_data['genres'].apply(lambda x:[i['name'].lower().strip() for i in x])

In [ ]:
merged_data['genres'].iloc[0]

['action', 'adventure', 'fantasy', 'science fiction']

mergeing cast,director,crew,genres

In [ ]:
merged_data['merged_description'] = merged_data['keywords'] + merged_data['cast'] + merged_data['director'] + merged_data['genres']

In [ ]:
merged_data['merged_description'].iloc[0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relation',
 'mind and soul',
 '3d',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'jamescameron',
 'jamescameron',
 'jamescameron',
 'action',
 'adventure',
 'fantasy',
 'science fiction']

In [ ]:
merged_data['merged_description'] = merged_data['merged_description'].apply(" ".join)

In [ ]:
merged_data['merged_description'].iloc[0]

'culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relation mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver jamescameron jamescameron jamescameron action adventure fantasy science fiction'

similarity using sbert model

In [ ]:
merged_data['merged_description'] = merged_data['merged_description'].apply(sbert_model.encode)

In [ ]:
merged_description = list(merged_data['merged_description'])

In [ ]:
def get_recommendation_based_keywords_director(title):
  similarity = []
  try:
    sim_input = merged_description[movie_name.index(title)]
  except:
    print("found")
    sim_input = " ".join(LemNormalize(title))
    sim_input = sbert_model.encode(sim_input)

  for count,compare in enumerate(merged_description):
    sim = cosine_similarity(sim_input,compare)
    temp = {'name':movie_name[count],'similarity':sim}
    similarity.append(temp)

  top_30 = (sorted(similarity,key=lambda d:d['similarity'],reverse=True))[:30]
  return top_30

In [ ]:
top_30 = get_recommendation_based_keywords_director('The Dark Knight')

In [ ]:
print(top_30)

[{'name': 'The Dark Knight', 'similarity': 1.0000001}, {'name': 'Batman Begins', 'similarity': 0.9478095}, {'name': 'The Dark Knight Rises', 'similarity': 0.9247143}, {'name': 'Batman & Robin', 'similarity': 0.91817456}, {'name': 'Batman', 'similarity': 0.90890455}, {'name': 'Dick Tracy', 'similarity': 0.89804876}, {'name': 'Suicide Squad', 'similarity': 0.88688654}, {'name': 'Kick-Ass 2', 'similarity': 0.88052416}, {'name': 'Kick-Ass', 'similarity': 0.88050324}, {'name': 'Dredd', 'similarity': 0.864202}, {'name': 'Batman Forever', 'similarity': 0.8590824}, {'name': 'Defendor', 'similarity': 0.85794616}, {'name': 'Spawn', 'similarity': 0.8512966}, {'name': 'Sherlock Holmes: A Game of Shadows', 'similarity': 0.8503746}, {'name': 'RoboCop 3', 'similarity': 0.8438541}, {'name': 'Sin City', 'similarity': 0.84010285}, {'name': 'Batman v Superman: Dawn of Justice', 'similarity': 0.83991706}, {'name': 'Repo Men', 'similarity': 0.8348486}, {'name': 'District B13', 'similarity': 0.8329487}, {'n

similarity using diffrent model(works more better)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity


In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(merged_data['merged_description'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
def get_recommendations_crew_cast_keyword_genre(title):
    idx = (merged_data[merged_data['original_title'] == title].index.values)[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return merged_data['original_title'].iloc[movie_indices]


In [ ]:
get_recommendations_crew_cast_keyword_genre('The Dark Knight').head(15)

119                          Batman Begins
3                    The Dark Knight Rises
428                         Batman Returns
210                         Batman & Robin
1196                          The Prestige
813                               Superman
1359                                Batman
1720                              Kick-Ass
1296                          Superman III
1033                              Insomnia
9       Batman v Superman: Dawn of Justice
1740                            Kick-Ass 2
299                         Batman Forever
14                            Man of Steel
96                               Inception
Name: original_title, dtype: object

#**Recommendation based on user using Collabrative Filtering**


Building the dataset

In [ ]:
user_data = pd.read_csv('/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /data/ratings_small.csv')

In [ ]:
movie_id=list(data['id']) 

In [ ]:
user_data.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

Data preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
user_enc = LabelEncoder()
user_data['user'] = user_enc.fit_transform(user_data['userId'].values)
n_users = user_data['user'].nunique()
item_enc = LabelEncoder()
user_data['movie'] = item_enc.fit_transform(user_data['movieId'].values)
n_movies = user_data['movie'].nunique()
user_data['rating'] = user_data['rating'].values.astype(np.float32)
min_rating = min(user_data['rating'])
max_rating = max(user_data['rating'])

In [ ]:
old_movie_id = list(user_data['movieId'])
new_movie_id = list(user_data['movie'])
new_to_orignal_movie_id = {new_movie_id[i]:old_movie_id[i] for i in range(len(old_movie_id))}
original_to_new_movie_id = {old_movie_id[i]:new_movie_id[i] for i in range(len(old_movie_id))}

In [ ]:
'''As for the new user id , the new user id is only -1 from old user id '''

'As for the new user id , the new user id is only -1 from old user id '

In [ ]:
movie_data = pd.read_csv('/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /data/movies_metadata.csv')
movie_data['genres'] = movie_data['genres'].apply(ast.literal_eval)
def extract_genre(genre_list):
  extracted=''
  for genre in genre_list:
    extracted+=('|'+genre['name'])
  return extracted
movie_data['extracted_genres'] = movie_data['genres'].apply(extract_genre)
movie_data['extracted_genres'].head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0     |Animation|Comedy|Family
1    |Adventure|Fantasy|Family
2              |Romance|Comedy
3        |Comedy|Drama|Romance
4                      |Comedy
Name: extracted_genres, dtype: object

In [ ]:
X = user_data[['user', 'movie']].values

In [ ]:
y = user_data['rating'].values 

In [ ]:
X,y = shuffle(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
n_factors = 50

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90003, 2), (10001, 2), (90003,), (10001,))

In [ ]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

Build Model

In [ ]:
def Recommender_model(n_users,n_movies,n_factors):
  user = Input(shape=(1,))
  u=Embedding(n_users,n_factors,embeddings_initializer='he_normal',embeddings_regularizer=l2(1e-6))(user)
  u=Reshape((n_factors,))(u)
   
  movie = Input(shape=(1,))
  m=Embedding(n_movies,n_factors,embeddings_initializer='he_normal',embeddings_regularizer=l2(1e-6))(movie)
  m=Reshape((n_factors,))(m)

  X = concatenate([u,m])
  X=Dropout(rate=0.2)(X)

  X = Dense(10,activation='relu')(X)
  X= Dropout(rate=0.2)(X)

  X=Dense(1,activation='sigmoid')(X)
  X = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(X) 

  model = Model(inputs=[user, movie], outputs=X)
  opt = Adam(lr=0.001)
  model.compile(loss='mean_squared_error', optimizer=opt)

  return model

  


In [ ]:
R_model=Recommender_model(n_users,n_movies,n_factors)

In [ ]:
R_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 50)        33550       input_8[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 50)        453300      input_9[0][0]                    
____________________________________________________________________________________________

In [ ]:
R_model.fit(X_train_array,y_train,validation_data=(X_test_array,y_test),epochs=8)

Epoch 1/8
2813/2813 [==============================] - 14s 5ms/step - loss: 1.0308 - val_loss: 0.7891
Epoch 2/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.7512 - val_loss: 0.7744
Epoch 3/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.7152 - val_loss: 0.7782
Epoch 4/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.6851 - val_loss: 0.7834
Epoch 5/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.6738 - val_loss: 0.7905
Epoch 6/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.6627 - val_loss: 0.7876
Epoch 7/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.6535 - val_loss: 0.7934
Epoch 8/8
2813/2813 [==============================] - 13s 5ms/step - loss: 0.6452 - val_loss: 0.7933


In [ ]:
Model.save(R_model,'/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /data/saved_model/R_model.h5')

Load Model

In [ ]:
R_model = load_model('/content/drive/MyDrive/my projects/reccomender system/Movie Recommender system /saved_model/R_model.h5')

Create Recommder function

In [ ]:
movie_ids=list(set(new_movie_id))

In [ ]:
def recommend_user(user_id):
  ids=np.ones(len(movie_ids)) * (user_id-1)
  predictions = R_model.predict([ids,np.array(movie_ids)])
  predictions_dictionary = [{'id':new_to_orignal_movie_id[movie_ids[i]],'prediction':predictions[i]} for i in range(len(movie_ids))]
  recommendations = (sorted(predictions_dictionary,key=lambda d:d['prediction'],reverse=True))[:30]
  print("=================Movies Rated By user=============")
  indexes =  list(movie_data[user_data['userId']==user_id].index.values)
  rated_movie_id = list(user_data.loc[indexes,'movieId'])
  for id in rated_movie_id:
    idx=list(movie_data[movie_data['id']==str(id)].index.values)
    if len(idx)==0:
      continue
    print(movie_data[['title','extracted_genres']].iloc[idx].values)
  print("=================Movie recommendation for user=====================")
  for part in recommendations:
    temp = part['id']
    idx = list(movie_data[movie_data['id']==str(temp)].index.values)
    if len(idx)==0:
      continue
    print(movie_data[['title','extracted_genres']].iloc[idx].values)

In [ ]:
recommend_user(2)

=================Movies Rated By user=============
[['The Dark' '|Horror|Thriller|Mystery']]
[['2001: A Space Odyssey' '|Science Fiction|Mystery|Adventure']]
[['Three Colors: Red' '|Drama|Mystery|Romance']]
[['Wings of Desire' '|Drama|Fantasy|Romance']]
[['48 Hrs.' '|Thriller|Action|Comedy|Crime|Drama']]
[['Lost in Translation' '|Drama']]
[["Ocean's Eleven" '|Thriller|Crime']]
[['Back to the Future Part II' '|Adventure|Comedy|Family|Science Fiction']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


[['Star Trek IV: The Voyage Home' '|Science Fiction|Adventure']]
[['A Clockwork Orange' '|Science Fiction|Drama']]
[['Lucky Number Slevin' '|Drama|Thriller|Crime|Mystery']]
[['Berlin: Symphony of a Great City' '|Documentary|History']]
[['Rebecca' '|Drama|Mystery']]
[['Man of Iron' '|Drama|History']]
[['Stand by Me' '|Crime|Drama']]
[['Pocketful of Miracles' '|Comedy|Drama']]
[['Live and Let Die' '|Adventure|Action|Thriller']]
[['Cat on a Hot Tin Roof' '|Drama|Romance']]
[['Lili Marleen' '|War|Drama|Romance']]
[['Contempt' '|Drama']]
[['Batman Begins' '|Action|Crime|Drama']]
[['The White Sound' '|Drama']]
[["Dave Chappelle's Block Party" '|Comedy|Documentary|Music']]
[['Terminator 3: Rise of the Machines' '|Action|Thriller|Science Fiction']]
[['The Science of Sleep' '|Comedy|Drama|Fantasy']]
[['Catwoman' '|Action|Crime']]
[['Grbavica: The Land of My Dreams' '|Drama|History']]
[['True Romance' '|Action|Thriller|Crime|Romance']]
[['Night on Earth' '|Comedy|Drama']]
[['Cockles and Muscles'